In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

import os
import re
import numpy as np
import matplotlib.pyplot as plt


In [3]:
 

# path_to_zip = tf.keras.utils.get_file('cornell_movie_dialogs.zip',
# origin='http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip',extract=True)

path_to_dataset = os.path.join(os.path.dirname("C:/code_test/data_ai"), "cornell movie-dialogs corpus")

path_to_movie_lines = os.path.join("C:\code_test\data_ai\cornell_movie_dialogs_extracted\cornell movie-dialogs corpus", "movie_lines.txt")
path_to_movie_conversations = os.path.join("C:\code_test\data_ai\cornell_movie_dialogs_extracted\cornell movie-dialogs corpus", "movie_conversations.txt")


<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:7: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:7: SyntaxWarning: invalid escape sequence '\c'
C:\Users\admin\AppData\Local\Temp\ipykernel_31240\1225956407.py:6: SyntaxWarning: invalid escape sequence '\c'
  path_to_movie_lines = os.path.join("C:\code_test\data_ai\cornell_movie_dialogs_extracted\cornell movie-dialogs corpus", "movie_lines.txt")
C:\Users\admin\AppData\Local\Temp\ipykernel_31240\1225956407.py:7: SyntaxWarning: invalid escape sequence '\c'
  path_to_movie_conversations = os.path.join("C:\code_test\data_ai\cornell_movie_dialogs_extracted\cornell movie-dialogs corpus", "movie_conversations.txt")


In [ ]:
# print("Path to extracted files:")
# print(os.path.dirname(path_to_zip))

Path to extracted files:


In [6]:
###  
#Maximum number of samples to preprocess
MAX_SAMPLES = 50000
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    #creating a space between a word and the punctuation following it
    #eg: "he is a boy." => "he is a boy ."
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    #replacing everything with space except  (a-z, A-Z, ".", "?", "!", ",")
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence


In [7]:
def load_conversations():
    # dictionary of line id to text
    id2line = {}
    with open(path_to_movie_lines, errors='ignore') as file:
        lines = file.readlines()
    for line in lines:
        parts = line.replace('\n', '').split(' +++$+++ ')
        id2line[parts[0]] = parts[4]

    # Tạo danh sách các cặp câu hỏi – trả lời từ movie_conversations.txt
    inputs, outputs = [], []
    with open(path_to_movie_conversations, 'r') as file:
         lines = file.readlines()
    for line in lines:
        parts = line.replace('\n', '').split(' +++$+++ ')
        # get conversation in a list of line ID
        conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]  # eval() để chuyển chuỗi dạng list thành list thực
        for i in range(len(conversation) - 1):
            inputs.append(preprocess_sentence(id2line[conversation[i]]))
            outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))
            if len(inputs) >= MAX_SAMPLES:
                return inputs, outputs
    return inputs, outputs
questions, answers = load_conversations()

In [10]:
print ('Sample question: {}'.format(questions[1]))
print ('Sample answer: {}'.format(answers[1]))

Sample question: well , i thought we d start with pronunciation , if that s okay with you .
Sample answer: not the hacking and gagging and spitting part . please .


In [12]:
# Build tokenizer using tfds for both questions and answers
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**15)

print("vocab size before: ",tokenizer.vocab_size)

# Define start and end token to indicate the start and end of a sentence
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print("start token id:", START_TOKEN)
print("end token id:", END_TOKEN)

# Vocabulary size plus start and end token
VOCAB_SIZE = tokenizer.vocab_size + 2
print("vocab size after:", VOCAB_SIZE)

vocab size before:  32574
start token id: [32574]
end token id: [32575]
vocab size after: 32576


In [13]:
# Maximum sentence length 
MAX_LENGTH = 40


# Tokenize, filter and pad sentences
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # tokenize sentence
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
    # check tokenized sentence max length
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # pad tokenized sentences
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs


questions, answers = tokenize_and_filter(questions, answers)

In [14]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# decoder input use the previous target as input 
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#######
for enc_dec_input, output in dataset.take(1):
    print("encoder input: ", enc_dec_input["inputs"])
    print("#" * 75)
    print("decoder input: ", enc_dec_input["dec_inputs"])
    print("#" * 75)
    print(output)


In [17]:
def scaled_dot_product_attention(query, key, value, mask):
  """Calculate the attention weights. """
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # scale matmul_qk
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask to zero out padding tokens
  if mask is not None:
    logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  output = tf.matmul(attention_weights, value)

  return output

In [18]:
  ### edit as slide
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)
  
  
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # scaled dot-product attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(
        scaled_attention, perm=[0, 2, 1, 3])

    # concatenation of heads
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # final linear layer
    outputs = self.dense(concat_attention)

    return outputs    
     